In [4]:
import os
import cv2
import torch
import pandas as pd
from torchvision import models, transforms
from PIL import Image

# Cargar el modelo pre-entrenado ResNet50
model = models.resnet50(pretrained=True)
model.eval()  # Poner el modelo en modo evaluación

# Preprocesamiento de imágenes para el modelo
preprocess = transforms.Compose([
    transforms.Resize((224, 224)),  # Redimensionar a 224x224
    transforms.ToTensor(),  # Convertir a tensor
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),  # Normalizar
])

# Cargar las etiquetas de ImageNet
with open('imagenet_classes.txt') as f:
    labels = [line.strip() for line in f.readlines()]

def preprocess_frame(frame):
    """
    Preprocesa un fotograma para que sea compatible con el modelo ResNet50.
    """
    frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)  # Convertir de BGR a RGB
    frame = Image.fromarray(frame)  # Convertir a PIL Image
    frame = preprocess(frame)  # Aplicar el preprocesamiento
    frame = frame.unsqueeze(0)  # Añadir dimensión del batch
    return frame

def detect_objects_in_frame(frame):
    """
    Detecta objetos en un fotograma utilizando el modelo ResNet50.
    Devuelve la clase más probable y la confianza.
    """
    with torch.no_grad():  # Desactivar el cálculo de gradientes
        outputs = model(frame)
    _, predicted = torch.max(outputs, 1)
    confidence = torch.nn.functional.softmax(outputs, dim=1)[0] * 100
    class_name = labels[predicted.item()]
    confidence = confidence[predicted.item()].item()
    return class_name, confidence

def process_video_frames(frames_dir):
    """
    Procesa todos los fotogramas de un vídeo y devuelve un DataFrame con los resultados.
    """
    results = []
    
    # Listar todos los fotogramas en el directorio
    frame_files = sorted(os.listdir(frames_dir), key=lambda x: int(x.split('.')[0]))
    
    for frame_file in frame_files:
        frame_path = os.path.join(frames_dir, frame_file)
        frame = cv2.imread(frame_path)
        
        if frame is not None:
            frame_number = int(frame_file.split('.')[0])
            preprocessed_frame = preprocess_frame(frame)
            class_name, confidence = detect_objects_in_frame(preprocessed_frame)
            
            results.append({
                'frame_number': frame_number,
                'class_name': class_name,
                'confidence': confidence
            })
    
    # Convertir los resultados a un DataFrame
    return pd.DataFrame(results)

# Ejemplo de uso
if __name__ == "__main__":
    # Directorio que contiene los fotogramas del vídeo
    frames_directory = r"C:\Users\Usuario\Desktop\GitHub\Grupo-Atrium\Deep Learning\frames"
    
    # Procesar los fotogramas y obtener el DataFrame
    results_df = process_video_frames(frames_directory)
    
    # Guardar los resultados en un archivo CSV
    results_df.to_csv('resultados_deteccion_pytorch.csv', index=False)
    
    print("Detección de objetos completada. Resultados guardados en 'resultados_deteccion_pytorch.csv'.")

FileNotFoundError: [Errno 2] No such file or directory: 'imagenet_classes.txt'